In [1]:
from random import Random
import numpy as np
import pandas as pd
from tva.situation import Situation
from tva.strategies import Strategies
import copy
from tva.schemes import Schemes, VotingScheme
from tva.voter import Voter

In [2]:
# Create the voting situation
situation = Situation(num_voters=4, num_candidates=4, seed=42)
display(situation)

# Apply the voting scheme to the situation
schemes = Schemes()
voting_scheme = VotingScheme.BORDA
schemes.print_results(situation, verbose=True)



Voter 0: ['A', 'C', 'D', 'B']
Voter 1: ['D', 'A', 'B', 'C']
Voter 2: ['C', 'A', 'D', 'B']
Voter 3: ['A', 'B', 'C', 'D']

Anti plurality: A {'A': 4, 'C': 3, 'D': 3, 'B': 2} 
Two voting: A {'A': 4, 'C': 2, 'D': 1, 'B': 1} 
Borda: A {'A': 10, 'C': 6, 'D': 5, 'B': 3}


In [30]:
situation = Situation(num_voters=4, num_candidates=4)
situation.voters[0].preferences = ["A","B","C","D"]
situation.voters[1].preferences = ["C","B","A","D"]
situation.voters[2].preferences = ["B","A","D","C"]
situation.voters[3].preferences = ["C","B","A","D"]
display(situation)

current_winner, scores = schemes.apply_voting_scheme(VotingScheme.BORDA, situation.voters, return_scores=True)
print("Borda:", current_winner, scores)

Voter 0: ['A', 'B', 'C', 'D']
Voter 1: ['C', 'B', 'A', 'D']
Voter 2: ['B', 'A', 'D', 'C']
Voter 3: ['C', 'B', 'A', 'D']

Borda: B {'A': 7, 'B': 9, 'C': 7, 'D': 1}


In [33]:
def swap(situation:Situation, voter_id:int, candidate1_index:int, candidate2_index:int, verbose=False):
    if verbose:
        print(f"Swapping {situation.voters[voter_id].preferences[candidate1_index]} and {situation.voters[voter_id].preferences[candidate2_index]}")
    situation.voters[voter_id].preferences[candidate1_index], situation.voters[voter_id].preferences[candidate2_index] = situation.voters[voter_id].preferences[candidate2_index], situation.voters[voter_id].preferences[candidate1_index]

def bury(situation:Situation, voter_index:int, verbose=False):
    original_preferences = situation.voters[voter_index].preferences
    # If the original winner is the first preference of the voter, return False
    original_winner, scores = schemes.apply_voting_scheme(VotingScheme.BORDA, situation.voters, return_scores=True)
    original_winner_index = original_preferences.index(original_winner) # type: ignore
    original_winner_score = original_preferences.index(original_winner) # type: ignore

    if verbose:
        print(original_preferences)
        print("Borda:", original_winner, scores)

    if original_winner_index == 0:
        return False
    return recursive_bury(situation, voter_index, [original_preferences], original_preferences, original_winner_score, verbose)

def recursive_bury(situation:Situation, voter_index:int, past_preferences:list, true_preferences:list, original_winner_score:int, verbose=False):
    starting_preferences = situation.voters[voter_index].preferences
    num_candidates = len(starting_preferences)
    modified_situation = copy.deepcopy(situation)

    # Save the original winner
    starting_winner, scores = schemes.apply_voting_scheme(VotingScheme.BORDA, situation.voters, return_scores=True)
    starting_winner_index = starting_preferences.index(starting_winner) # type: ignore
    # Initialize the current winner
    
    # Keep moving the winner to the right until it chantes the winner or reaches the end of the loop
    for i in range(starting_winner_index+1, num_candidates):
        swap(modified_situation, voter_index, i-1, i, verbose)
        new_preferences = modified_situation.voters[voter_index].preferences
        if verbose:
            print(new_preferences)

        if new_preferences in past_preferences:
            if verbose:
                print("Loop detected")
            continue
        past_preferences.append(copy.deepcopy(new_preferences))

        current_winner, scores = schemes.apply_voting_scheme(VotingScheme.BORDA, modified_situation.voters, return_scores=True)
        current_winner_score = true_preferences.index(current_winner)
        
    
        if verbose:
            print("Borda:", current_winner, scores)
        if current_winner != starting_winner:
            if current_winner_score < original_winner_score:
                if verbose:
                    print("Found a winning strategy!")
                return True
            else:
                if verbose:
                    print("Winner changed")
                found_winning_strategy = recursive_bury(modified_situation, voter_index, past_preferences, true_preferences, original_winner_score)
                if found_winning_strategy:
                    return True

    # If finished the loop, then the winner is still the same, so return False
    if verbose:
        print("Left recursion")
    return False

print(situation)

print("Will use bury?",bury(situation, 0))


Voter 0: ['A', 'B', 'C', 'D']
Voter 1: ['C', 'B', 'A', 'D']
Voter 2: ['B', 'A', 'D', 'C']
Voter 3: ['C', 'B', 'A', 'D']

Will use bury? True


Create 1000 situations and check for what fraction of them a strategy is good

We do experiments for each voting scheme separately.<br>
TODO: We need to save, what type of strategy has won this time?

In [1]:
strategies = Strategies()
voting_scheme = VotingScheme.BORDA
situation = Situation(num_voters=5, num_candidates=4)
situation

NameError: name 'Strategies' is not defined

In [33]:
strategies = Strategies()
voting_scheme = VotingScheme.BORDA
nr_repetitions = 1000
num_voters = 4

strategy_counter = 0
for i in range(nr_repetitions):
    situation = Situation(num_voters=num_voters, num_candidates=4)
    # Check if at least one voter has a good strategy
    for voter_index in range(num_voters):
        if strategies.is_any_strategy_good(situation, voter_index, voting_scheme):
            strategy_counter += 1
            break
        
print(strategy_counter/nr_repetitions)

0.877


Plurality cannot be affected by strategic voting because that would require a voter to vote for a candidate they prefer less than their favorite candidate.